In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_5월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(862586, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
862586


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,"[아테네, 이외, 에도, 그리스, 는, 국토, 전역, 이, 문화유산, 이어서, 어느...","[('아테네', 'Noun'), ('이외', 'Noun'), ('에도', 'Josa..."
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0,"[나, 는, 그, 중, 에서도, 학창, 시절, 부터, 익히, 들었던, 펠로폰네소스,...","[('나', 'Noun'), ('는', 'Josa'), ('그', 'Determin..."
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0,"[펠로폰네소스, 지역, 은, 그리스, 최초, 의, 문화인, 미케네, 문명, 이, 싹...","[('펠로폰네소스', 'Noun'), ('지역', 'Noun'), ('은', 'Jo..."
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0,"[미케네, 는, 기원전, 1500년, 부터, 기원전, 1200년, 까지, 소아시아,...","[('미케네', 'Noun'), ('는', 'Josa'), ('기원전', 'Noun..."
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0,"[미케네, 인들, 은, 매우, 전투, 적, 이어서, 주변, 지역, 을, 비롯, 해,...","[('미케네', 'Noun'), ('인들', 'Josa'), ('은', 'Noun'..."
...,...,...,...,...
862581,0 접기 펴기 멋있는 난타공연 난타 공연 너무 멋짐,19230,"[0, 접기, 펴기, 멋있는, 난타, 공연, 난타, 공연, 너무, 멋짐]","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb..."
862582,와 흥이 절로 나고 신났음 ㅋ,19230,"[와, 흥이, 절로, 나고, 신났음, ㅋ]","[('오다', 'Verb'), ('흥이', 'Noun'), ('절로', 'Noun'..."
862583,하이라이트 소리지름 주의 LED공연 하이라이트 재생 14 좋아요,19230,"[하이라이트, 소리, 지름, 주의, LED, 공연, 하이라이트, 재생, 14, 좋아요]","[('하이라이트', 'Noun'), ('소리', 'Noun'), ('지름', 'No..."
862584,0 접기 펴기 LED공연 하이라이트 정말 잊을수 없는 공연이었고 못 보고 갔으면 정...,19230,"[0, 접기, 펴기, LED, 공연, 하이라이트, 정말, 잊을수, 없는, 공연, 이...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(146002, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '등', '때', '우리', '말', '문화', '더', '생각', '길', '나', '내', '시간', '나무', '사진', '위', '전', '제', '안', '중', '공원', '의', '아이', '마을', '때문', '일', '여행', '공간', '작품', '집', '하나', '날', '저', '층', '꽃', '서울', '모습', '위해', '개', '및', '시작', '카페', '체험', '한국', '앞', '마음']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 862586/862586 [00:06<00:00, 130953.22it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

23
23


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 862586/862586 [09:33<00:00, 1504.50it/s] 


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [11]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/5월/단어동시출현_딕셔너리_5월.csv", encoding='UTF-8')

In [12]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 153326521/153326521 [17:53<00:00, 142775.40it/s]


(153326521, 3)


,term1,term2,freq
0,대,문화유산,260
1,대,전역,105
2,대,향기,90
3,는,대,7330
4,국토,대,64


In [13]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/5월/단어동시출현_df_5월.csv", index=False, encoding='UTF-8')

In [14]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 153326521/153326521 [30:48<00:00, 82944.75it/s]


,term1,term2,freq
0,문화,수,6417
1,문화,이,16209
2,문화,의,17581
3,수,이,60490
4,수,의,52294
...,...,...,...
1220,개,카페,450
1221,때문,아이,568
1222,때문,마을,361
1223,여행,작품,259


In [15]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/5월/단어동시출현_df_상위50_5월.csv", index=False, encoding='UTF-8')

In [16]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1548.54it/s]


In [17]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,등,때,우리,말,문화,더,생각,길,나,내,시간,나무,사진,위,전,제,안,중,공원,의,아이,마을,때문,일,여행,공간,작품,집,하나,날,저,층,꽃,서울,모습,위해,개,및,시작,카페,체험,한국,앞,마음
것,0.0,24588.0,22893.0,88799.0,8858.0,13864.0,7279.0,11687.0,10249.0,10923.0,4969.0,8894.0,10179.0,3875.0,11912.0,9751.0,5650.0,4778.0,3662.0,6502.0,6025.0,4651.0,7025.0,5986.0,1782.0,65164.0,3605.0,2108.0,7186.0,7560.0,2513.0,3014.0,3607.0,3484.0,5030.0,3604.0,4867.0,1347.0,2485.0,1726.0,3748.0,4951.0,2843.0,1901.0,3874.0,1594.0,1660.0,2710.0,3806.0,4874.0
수,24588.0,0.0,13744.0,60490.0,11515.0,8947.0,9889.0,7417.0,6912.0,6112.0,6417.0,6957.0,6228.0,4547.0,8680.0,6452.0,6207.0,4908.0,4121.0,5232.0,5212.0,4462.0,5032.0,5003.0,3498.0,52294.0,4334.0,2707.0,5294.0,5306.0,3027.0,5764.0,3729.0,2780.0,3396.0,2691.0,3153.0,2524.0,2815.0,2341.0,3968.0,3441.0,3112.0,3181.0,2745.0,2456.0,4388.0,2486.0,3202.0,3503.0
그,22893.0,13744.0,0.0,50301.0,7733.0,9794.0,4808.0,8113.0,6102.0,7165.0,3100.0,4640.0,5037.0,3002.0,7401.0,5956.0,3143.0,3225.0,2141.0,4700.0,4236.0,3310.0,4620.0,5291.0,1039.0,47794.0,1949.0,1521.0,4246.0,4686.0,1420.0,1889.0,2835.0,2580.0,3143.0,2606.0,2511.0,949.0,1568.0,1256.0,2872.0,3243.0,1962.0,1232.0,2646.0,957.0,751.0,1712.0,2756.0,3083.0
이,88799.0,60490.0,50301.0,0.0,41743.0,36233.0,26927.0,30524.0,21216.0,22645.0,16209.0,20944.0,21796.0,14857.0,28383.0,21951.0,18031.0,16858.0,14881.0,16171.0,18500.0,14660.0,18742.0,17230.0,9174.0,174150.0,12240.0,9609.0,15695.0,16402.0,8204.0,12422.0,11064.0,11435.0,11790.0,13964.0,13687.0,6761.0,10607.0,7020.0,12972.0,11490.0,10880.0,7015.0,11557.0,7902.0,6998.0,8325.0,11500.0,12156.0
곳,8858.0,11515.0,7733.0,41743.0,0.0,4365.0,4114.0,4111.0,2570.0,2004.0,3632.0,3105.0,2816.0,3868.0,3743.0,2999.0,3300.0,2931.0,3422.0,2238.0,2966.0,2602.0,2806.0,3484.0,3307.0,23651.0,3240.0,2956.0,1730.0,1960.0,3758.0,2791.0,978.0,2073.0,1806.0,1768.0,2511.0,1313.0,1867.0,2067.0,2045.0,1646.0,1733.0,1017.0,1567.0,2475.0,2178.0,1255.0,1860.0,1739.0
사람,13864.0,8947.0,9794.0,36233.0,4365.0,0.0,2354.0,4616.0,3699.0,4736.0,2093.0,3264.0,4280.0,1983.0,5527.0,3949.0,2223.0,1958.0,2132.0,2404.0,2205.0,1637.0,3190.0,2443.0,890.0,24508.0,1044.0,1280.0,2527.0,3255.0,971.0,1261.0,989.0,1663.0,1853.0,1765.0,2080.0,555.0,1039.0,826.0,1763.0,2000.0,1177.0,562.0,1464.0,830.0,512.0,1021.0,1792.0,2410.0
등,7279.0,9889.0,4808.0,26927.0,4114.0,2354.0,0.0,2668.0,1719.0,1961.0,4258.0,1662.0,1279.0,1643.0,3567.0,2359.0,1873.0,2654.0,1475.0,3124.0,3605.0,2936.0,1726.0,2645.0,1876.0,26863.0,1242.0,1535.0,1330.0,1969.0,1102.0,2360.0,2006.0,1188.0,1074.0,1012.0,1029.0,1154.0,1361.0,1628.0,1407.0,1921.0,2447.0,3996.0,1424.0,1019.0,2372.0,2019.0,1136.0,781.0
때,11687.0,7417.0,8113.0,30524.0,4111.0,4616.0,2668.0,0.0,3402.0,3549.0,1587.0,2805.0,3384.0,1777.0,4747.0,3724.0,2293.0,1836.0,1835.0,2011.0,2492.0,2190.0,2879.0,2228.0,780.0,20380.0,1529.0,884.0,2029.0,2765.0,1271.0,903.0,907.0,1829.0,1714.0,1872.0,1921.0,540.0,1138.0,868.0,1564.0,1544.0,1157.0,618.0,1748.0,721.0,536.0,955.0,1587.0,1799.0
우리,10249.0,6912.0,6102.0,21216.0,2570.0,3699.0,1719.0,3402.0,0.0,2974.0,2301.0,2526.0,2915.0,1428.0,3264.0,2476.0,1946.0,1480.0,1294.0,1905.0,1751.0,1206.0,2402.0,1687.0,532.0,19713.0,1666.0,712.0,1672.0,2134.0,852.0,857.0,852.0,1967.0,1408.0,1395.0,1157.0,505.0,802.0,600.0,1091.0,1612.0,939.0,549.0,1114.0,599.0,694.0,909.0,1350.0,1686.0
말,10923.0,6112.0,7165.0,22645.0,2004.0,4736.0,1961.0,3549.0,2974.0,0.0,1376.0,1999.0,2560.0,1092.0,3951.0,3323.0,1226.0,1234.0,743.0,1829.0,1718.0,1699.0,2297.0,1846.0,416.0,18349.0,980.0,537.0,1524.0,2103.0,560.0,570.0,662.0,1017.0,1240.0,1081.0,1284.0,282.0,656.0,492.0,873.0,1233.0,816.0,661.0,1014.0,331.0,367.0,762.0,1193.0,1368.0


In [18]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/5월/공출현빈도_df_5월.csv", encoding='UTF-8')